# Downloading then processing LiDAR data
In this notebook we will:
* Go through the process of downloading LiDAR data from OpenTopography using their APIs
* Do some basic processing on the downloaded LiDAR using PDAL

In [ ]:
import urllib
import pathlib
import requests
import boto3
import botocore
import botocore.client
import geopandas
import shapely
import pdal
import json
import rioxarray
import matplotlib
import matplotlib.pyplot

# Downloading LiDAR data
We will be using the OpenTopography APIs where we can and downloading LiDAR within an area of interest.

1. Define region of interest
2. Download tile map
3. Select tiles to download (selection within boundary)
4. Download tiles

## 1. Create region of interest

In [ ]:
horizontal_crs = 2193 # NZTM2000 - EPSG:2193
vertical_crs = 7839 # NZVD2016 - EPSG:7839
x0 = 1765000; y0 = 5460000; x1 = 1765400; y1 = 5460500
boundary = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
boundary = geopandas.GeoDataFrame(geometry=[boundary], crs=horizontal_crs)
boundary.to_file(pathlib.Path().cwd() / "boundary.geojson")

In [ ]:
#boundary.plot()

## 2. Download tile map
* Use OpenTopography 'otCatalog' API to search for datasets within ROI
* Select one and download tilemap within the boundary
* Get tiles within ROI
* Download the tiles within the RIO
### Use OpenTopography 'otCatalog' API to search for datasets within ROI

In [ ]:
OT_CRS = "EPSG:4326"
SCHEME = "https"
NETLOC_API = "portal.opentopography.org"
PATH_API = "/API/otCatalog"

In [ ]:
bounds = boundary.geometry.to_crs(OT_CRS).bounds
api_query = {
    "productFormat": "PointCloud",
    "minx": bounds['minx'].min(), "miny": bounds['miny'].min(), "maxx": bounds['maxx'].max(), "maxy": bounds['maxy'].max(),
    "detail": False, "outputFormat": "json", "inlcude_federated": True
}

data_url = urllib.parse.urlunparse((SCHEME, NETLOC_API, PATH_API, "", "", ""))

response = requests.get(data_url, params=api_query, stream=True)
response.raise_for_status()
json_response = response.json()

In [ ]:
#json_response

In [ ]:
for dataset in json_response['Datasets']:
    print(f"dataset name: {dataset['Dataset']['name']}, alternateName: {dataset['Dataset']['alternateName']}")

### Select one and download tilemap within the boundary
Hardcode AWS location and create client

In [ ]:
NETLOC_DATA = "opentopography.s3.sdsc.edu"
OT_BUCKET = "pc-bulk"

PATH_DATA = "/minio/pc-bulk/"

In [ ]:
ot_endpoint_url = urllib.parse.urlunparse((SCHEME, NETLOC_DATA, "", "", "", ""))
client = boto3.client('s3', endpoint_url=ot_endpoint_url, config=botocore.config.Config(signature_version=botocore.UNSIGNED))

Select dataset to download, and download

In [ ]:
dataset_prefix = json_response['Datasets'][0]['Dataset']['alternateName']
file_prefix = f"{dataset_prefix}/{dataset_prefix}_TileIndex.zip"

local_file_path = pathlib.Path().cwd() / dataset_prefix
local_tile_file = pathlib.Path().cwd() / file_prefix
local_file_path.mkdir(parents=True, exist_ok=True)

client.download_file(OT_BUCKET, file_prefix, str(local_tile_file))

### Get tiles within ROI
Open tile file and trim to the boundary

In [ ]:
tile_info = geopandas.read_file(local_tile_file)
tile_info = tile_info.to_crs(boundary.crs)

In [ ]:
#tile_info

In [ ]:
tile_info = geopandas.sjoin(tile_info, boundary)
tile_info = tile_info.reset_index(drop=True)

In [ ]:
tile_info

### Download the tiles within the RIO

In [ ]:
for url in tile_info['URL']:
    # drop the OT_BUCKET from the URL path to get the file_prefix
    file_prefix = pathlib.Path(*pathlib.Path(urllib.parse.urlparse(url).path).parts[2:])
    
    print(f"Downloading file: {file_prefix}")
    try:
        client.download_file(OT_BUCKET, str(file_prefix.as_posix()), str(local_file_path / file_prefix.name))
    except botocore.exceptions.ClientError as e:
        f"An error occured when trying to download {file_prefix}. The error is {e}"

# Processing LiDAR with PDAL
We will use PDAL to:
* Load in a LiDAR tile
* Reproject to CRS
* Filter within the ROI
* Produce a DEM

By constructing then running a PDAL pipeline

In [ ]:
lidar_file = str(local_file_path / file_prefix.name)

In [ ]:
lidar_file

### Pipeline instruction: Load in a LiDAR tile

In [ ]:
pdal_pipeline_instructions = [{"type":  "readers.las", "filename": str(lidar_file)}]

### Pipeline instruction: Reproject to CRS

In [ ]:
pdal_pipeline_instructions.append(
    {"type": "filters.reprojection", "out_srs": f"EPSG:{horizontal_crs}+{vertical_crs}"})

## Pipeline instruction: Filter within the ROI

In [ ]:
pdal_pipeline_instructions.append(
    {"type": "filters.crop", "polygon": str(boundary.loc[0].geometry)})

### Pipeline instruction: Produce a DEM

In [ ]:
dem_file = local_file_path / "dem.tif"
pdal_pipeline_instructions.append(
    {"type": "writers.gdal", "resolution": 10, "radius": 14.14, "filename": str(dem_file)})

## Execute pipeline

In [ ]:
pdal_pipeline_instructions

In [ ]:
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()

## Load in the results
Look at the contents of the pipeline output

In [ ]:
#pdal_pipeline.arrays
#pdal_pipeline.metadata

Look at the DEM

In [ ]:
with rioxarray.rioxarray.open_rasterio(dem_file, masked=True) as dem:
    dem.load()

In [ ]:
#dem

In [ ]:
layer_index = 1
dem_layer = dem.sel(band=layer_index)
dem_layer.attrs['long_name'] = dem_layer.attrs['long_name'][layer_index]
dem_layer.attrs['units'] = dem_layer.attrs['units'][layer_index]

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 10))
dem_layer.plot(ax=ax)